# Lectura de la Información
En este notebook nos enfocaremos en extraer la data de los archivos comprimidos y unificar las fuentes de datos. Muchas de los métodos/funciones utilizadas, serán cargadas de un archivo utils.py previamente generado para evitar sobrecarga de código en este notebook.

### Cargar librerías necesarias

In [2]:
from utils import *
import shutil
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

### Listar archivos comprimidos y extraer archivos

In [4]:
files=[i for i in os.listdir("../") if i.endswith(".zip")] ## ../ hace referencia a un nivel atrás al actual, en donde se encuentran los archivos .zip
_=[shutil.unpack_archive(f'../{i}', f'../data/') for i in files]

In [5]:
##extraer paths de cada archivo csv
filepaths=get_csv_filepaths([i.replace(".zip","") for i in files])

### Para leer las fuentes de datos se definirán grupos de archivos denotados por la variable filegroups

In [7]:
filegroups=['personajuridica','personanatural','produccion','recaudo','siniestro']     
data={}
for filegp in filegroups:
    data[filegp]=read_by_filegroup(filegp,filepaths,save_output=True)

In [9]:
##leer archivos pickle
pn=pd.read_pickle("personanatural.pickle",compression="gzip")